In [ ]:
import ee

# Forzamos el modo notebook para evitar el error de client_id
ee.Authenticate(auth_mode='notebook')

try:
    # Reemplaza con tu ID de proyecto registrado
    ee.Initialize(project='ee-valentinacamposo')
    print("✅ ¡Conectado correctamente!")
except Exception as e:
    print(f"❌ Error al inicializar: {e}")

In [16]:
# --- 1. FORZAR BÚSQUEDA POR NOMBRE EXACTO ---
# Intentamos con el dataset de límites administrativos nivel 2
comunas = ee.FeatureCollection("FAO/GAUL/2015/level2")

# Filtramos Pudahuel (asegurándonos que sea el de Chile)
pudahuel_limite = comunas.filter(ee.Filter.And(
    ee.Filter.eq('ADM2_NAME', 'Pudahuel'),
    ee.Filter.eq('ADM1_NAME', 'Region Metropolitana de Santiago')
))

# VERIFICACIÓN DE SEGURIDAD
count = pudahuel_limite.size().getInfo()

if count > 0:
    print(f"✅ ¡ÉXITO! Se encontró el polígono oficial de Pudahuel.")
    aoi = pudahuel_limite.geometry()
else:
    print("❌ El polígono sigue sin aparecer en el dataset GAUL.")
    print("Usando coordenadas del buscador de Google para crear el área...")
    # Estas coordenadas cubren el área que muestra el buscador de GEE
    aoi = ee.Geometry.Rectangle([-70.93, -33.48, -70.74, -33.28])
    pudahuel_limite = ee.FeatureCollection([ee.Feature(aoi)])

# --- 2. EXPORTACIÓN FORZADA ---
years = [2017, 2019, 2021, 2024]

for year in years:
    start = f'{year}-01-01'
    end = f'{year}-05-30' # Ampliamos el rango para asegurar imágenes

    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start, end)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(mask_clouds_s2)
    )

    if collection.size().getInfo() > 0:
        composite = collection.median().clip(pudahuel_limite)
        
        # Usamos el Bounding Box para la exportación
        export_region = aoi.bounds().getInfo()['coordinates']

        task = ee.batch.Export.image.toDrive(
            image=composite.select(['B2','B3','B4','B8','B11','B12']),
            description=f'sentinel2_pudahuel_FINAL_{year}',
            folder='cambio_urbano_pudahuel',
            region=export_region, 
            scale=10,
            maxPixels=1e9
        )
        task.start()
        print(f'🚀 Tarea enviada para {year}. REVISA TU DRIVE EN 10 MINUTOS.')

❌ El polígono sigue sin aparecer en el dataset GAUL.
Usando coordenadas del buscador de Google para crear el área...
🚀 Tarea enviada para 2017. REVISA TU DRIVE EN 10 MINUTOS.
🚀 Tarea enviada para 2019. REVISA TU DRIVE EN 10 MINUTOS.
🚀 Tarea enviada para 2021. REVISA TU DRIVE EN 10 MINUTOS.
🚀 Tarea enviada para 2024. REVISA TU DRIVE EN 10 MINUTOS.
